In [0]:
import pandas as pd
import matplotlib.pyplot as plt
import random
import numpy as np
from tqdm import tqdm
from scipy.spatial import distance
import math
import operator
import time
from tqdm import tqdm

import requests

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
data = pd.read_csv("/content/drive/My Drive/Decision_Models/cities.csv")

In [0]:
def telegram_bot_sendtext(bot_message):
    
    bot_token = '937044171:AAGGDmJcUTkkvDaOwcKrnHUyj9XUND7d52A'
    bot_chatID = '166942243'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message

    response = requests.get(send_text)

    return response.json()

In [0]:
def sieve_of_eratosthenes(n):
    primes = [True for i in range(n+1)] 
    primes[0] = False
    primes[1] = False 
    for i in range(2,int(np.sqrt(n)) + 1):
        if primes[i]:
            k = 2
            while i*k <= n:
                primes[i*k] = False
                k += 1
    return(primes)
prime_cities = sieve_of_eratosthenes(max(data["CityId"]))

In [0]:
def total_distance_v2(path):
    prev_city = path[0]
    total_distance = 0
    step_num = 1
    for city_num in path[1:]:
        next_city = city_num
        total_distance = total_distance + \
            np.sqrt(pow((data["X"][city_num] - data["X"][prev_city]),2) + pow((data["Y"][city_num] - data["Y"][prev_city]),2)) * \
            (1+ 0.1*((step_num % 10 == 0)*int(not(prime_cities[prev_city]))))
        prev_city = next_city
        step_num = step_num + 1
    return total_distance

# Algoritmo Genetico

In [0]:
class City:
    def __init__(self, x, y, id):
        self.x = x
        self.y = y
        self.id = id
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + "," + str(self.id) + ")"

In [0]:
##Fitness corretta

class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance == 0:
            pathDistance = 0
            step_number = 2
            start = City(x=data.X[0], y=data.Y[0], id=data.CityId[0])
            pathDistance = start.distance(self.route[0])
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = City(x=data.X[0], y=data.Y[0], id=data.CityId[0])
                pathDistance += fromCity.distance(toCity)* (1+ 0.1*((step_number % 10 == 0)*int(not(prime_cities[fromCity.id]))))
                step_number += 1
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [0]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [0]:
## Modifica per usare prima route come l'ordine decisio inizialmente (modifico il ciclo for e population all'inizio)

def initialPopulation(popSize, cityList):
    population = [cityList]

    for i in range(1, popSize):
        population.append(createRoute(cityList))
    return population

In [0]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [0]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [0]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [0]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [0]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        child = breed(pool[i], pool[len(matingpool)-i-1])
        children.append(child)
    return children

In [0]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [0]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        mutatedInd = mutate(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [0]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [0]:
def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
    
    pop = initialPopulation(popSize, population)
    progress = []
    initial_distance = 1 / rankRoutes(pop)[0][1]
    progress.append(initial_distance)
    print("Initial distance: " + str(initial_distance))
    telegram_bot_sendtext("Initial distance: " + str(initial_distance))
    
    for i in tqdm(range(0, generations)):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    final_distance = 1 / rankRoutes(pop)[0][1]
    print("Final distance: " + str(final_distance))
    telegram_bot_sendtext("Final distance: " + str(final_distance))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()
    return bestRoute

# SA


In [0]:
def generaPath(path): #fz che restituisce un path randomico dove primo elemento e ultimo coincidono (0)
  zero=[0]
  path=list(path)
  path= list(set(path).difference(set(zero)))
  path_inter=random.sample(path,len(path))
  path_inter.insert(0,0)
  return(path_inter)

In [0]:
nn = nearest_neighbour(data_10)
distance=total_distance(nn)
nn, distance

In [0]:
def SimulatedAnnealing(path_current,temp_itermax,temp_start,i_max): #path_current sarà valore migliore
  temp_min=0.1
  temp_iter=0
  i=0
  distance_current = total_distance(path_current)
  print("Distanza: "+str(distance_current))

  while temp_min<=temp_start:
    if (i>=i_max):
      break

    if temp_iter>=temp_itermax:
      temp_start=temp_start*0.5
      temp_iter=0
    print("Temperatura: "+str(temp_start))
    path_new=generaPath(path_current)
    distance_new=total_distance(path_new)
    i=i+1
    temp_iter=temp_iter+1
    print("Distanza: "+str(distance_new))

    if distance_new< distance_current:
      print("Accetto con probabilità 1")
      path_current=path_new
      distance_current = distance_new
    else:
      x=(distance_current-distance_new)/temp_start
      prob=math.exp(x)
      number_unif = np.random.uniform(low=0,high=1,size=1)
      print("Valore random: "+str(number_unif)+" | "+"Valore probabilità: "+str(prob))
      if number_unif< prob:
        print("Accetto con probabilità: "+str(prob))
        path_current=path_new
        distance_current = distance_new
      else:
        print("Rifiuto")
  print("Distanza finale: "+str(distance_current))
  return(path_current)

In [0]:
ottimo = SimulatedAnnealing(path_iniziale,100,1000,1000)
print("Migliore path: "+str(ottimo))

# 2-opt

In [0]:
## La distanza utilizzata è quella che considera la penalità sulla sequenza complessiva,
##  perchè il calcolo della distanza euclidea localmente non porta a miglioramenti
## (Non tiene conto delle penalità)

def runTS(sequenza):
        N = len(sequenza)
        migliore = total_distance_v2(sequenza)
        for i in range(N-3):
            for j in range(i+2,N-1):
                    newT = np.copy(sequenza)
                    
                    newT[i+1] = sequenza[j]
                    ind = i +2
                    for k in range(j-1,i,-1):
                        newT[ind] = sequenza[k]
                        ind += 1
                    newT_D = total_distance_v2(newT)
                        

                    if(newT_D < migliore):
                        sequenza = np.copy(newT)
                        migliore = np.copy(newT_D) 
                        runTS(sequenza)
        return migliore, sequenza